<a href="https://colab.research.google.com/github/paritoshk/ToughLeaf/blob/main/Tough_Leaf_Master_NLP_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Finetune GPT-2 on Reddit Data

by [Max Woolf](http://minimaxir.com)

A variant of the [default notebook](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) optimized for short-form titles. It is recommended to be familiar with that notebook before using this one.

This example uses a CSV export of Reddit data via BigQuery (see this post for more information).


In [2]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [1]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7 MB 9.8 kB/s 
     |████████████████████████████████| 3.2 MB 44.7 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 367 kB 43.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you

## GPU

In [3]:
!nvidia-smi

Thu Sep 23 19:14:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

The default query returns 1.3MB of data, so probably should only use `124M` GPT-2 to finetune. If working with more Reddity data, then migrate to `355M`.

In [5]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 208Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 1.64Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 247Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:46, 10.6Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 471Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.92Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.93Mit/s]


## Mounting Google Drive

In [6]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [7]:
import pandas as pd

In [10]:
df1 = pd.read_excel(r'/content/drive/MyDrive/ToughLeafDatasets/MBE,  WBE - Directory_2021-02-21.xlsx')

In [12]:
df1.head()

,Company Name,DBA Name,Owner First,Owner Last,Physical Address,City,State,Zip,Mailing Address,City.1,State.1,Zip.1,Phone,Fax,Email,Website,Agency,Certification Type,Certified,Capability,Work Districts/Regions,Industry,Business Size,General Location,Location,Commodity Codes
0,1 Accord Services Inc.,NaN,Helen,Washington,2331 Main Street,Buffalo,NY,14214,PO BOX 922,Buffalo,NY,14205,716-602-2404,716-675-7280,helen@1accordservices.com,http://www.1accordservices.com,NYS,MBE,2016-10-25,BUILDING MAINTENANCE; CARPET CLEANING; CLEANIN...,All work districts/regions,Services Consultants,"$100,000 - $499,000",Upstate New York,Western NY,5617 - Services to Buildings and Dwellings; 56...
1,1 Call Building Maintenance Corp,NaN,Lorris,Alleyne,946 Atlantic Ave.,Brooklyn,NY,11238,946 Atlantic Ave.,Brooklyn,NY,11238,347-469-0806,516-541-7720,lorrisalleyne1@gmail.com,http://http:www.1callbuildingmaintenance.com,NYS,MBE,2015-10-26,Our Services include Office Cleaning Carpet cl...,Long Island; Mid-Hudson; NYC,Services Consultants,"$100,000 - $499,000",Downstate New York,NYC,561720 - Janitorial services; 91009 - Carpet C...
2,1-800 Mr. Rubbish inc,NaN,Abeir,Saleh,1033 86th street,Brooklyn,NY,11228,1033 86th street,Brooklyn,NY,11228,800-677-8224,973-732-9145,contactus@1800mrrubbish.com,http://www.1800mrrubbish.com,NYS,WBE,2019-01-31,1-800 Mr. Rubbish prides itself on its zero wa...,All work districts/regions,Construction Consultants,"$500,000 - $999,999",Downstate New York,NYC,562111 - Recyclable material collection servic...
3,10 November LLC,NaN,Alfred,Reyes,"303 Wolfs Lane, Suite 1",Pelham,NY,10803,"303 Wolfs Lane, Suite 1",Pelham,NY,10803,914-885-1515,914-885-0074,a.reyes@10novgroup.com,http://www.10novgroup.com,NYS,MBE,2018-08-07,"Carpentry, masonry, concrete, finishing, repai...",Long Island; NYC,Construction,"$100,000 - $499,000",Downstate New York,NYC,425120 - Wholesale Trade Agents and Brokers; 9...
4,1020 Let's Go!,NaN,Karen,Eason,"550 Manor Road, #141148",Staten Island,NY,10314,"550 Manor Road, #141148",Staten Island,NY,10314,718-873-7599,NaN,ke@1020letsgo.com,1020letsgo.com,NYS,MBE,2019-01-22,Assist/teach startup and emerging entrepreneur...,All work districts/regions,Services Consultants,"Less than $100,000",Downstate New York,NYC,541611 - Administrative Management and General...


In [13]:
commodity_list_train = list(df1['Commodity Codes'])
capablity_list_train = list(df1['Capability'])

In [15]:
import re
commoditytest_split_words = [] 
for i in commodity_list_train:
    i = str(i)
    i = re.sub(r"(^|\W)\d+", " ", i)
    i = re.sub(r"-", " ", i)
    commoditytest_split_words.append(i)
capablitytest_split_words = [] 
for i in capablity_list_train:
    i = str(i)
    i = re.sub(r"(^|\W)\d+", " ", i)
    i = re.sub(r"-", " ", i)
    capablitytest_split_words.append(i)

str

## Uploading a Text File to be Trained to Colaboratory

A single-column CSV is expected.

In [ ]:
file_name = "results-20190831-145056.csv"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

Providing a single-column CSV will automatically add `<|startoftext|>` and `<|endoftext|>` tokens appropriately.

Short form text is more likely to overfit, so train it with fewer steps than you would for longform content.

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='774M',
              steps=500,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=100
              )

W0902 01:24:59.877743 139841164683136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/sample.py:17: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0902 01:25:14.827409 139841164683136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading checkpoint models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 30.39it/s]

Loading dataset...


dataset has 518272 tokens
Training...
[10 | 28.56] loss=1.72 avg=1.72
[20 | 52.24] loss=1.65 avg=1.69
[30 | 77.31] loss=1.61 avg=1.66
[40 | 101.84] loss=1.60 avg=1.64
[50 | 125.96] loss=1.58 avg=1.63
[60 | 150.46] loss=1.56 avg=1.62
[70 | 175.09] loss=1.40 avg=1.59
[80 | 199.58] loss=1.46 avg=1.57
[90 | 224.08] loss=1.33 avg=1.54
[100 | 248.60] loss=1.48 avg=1.53
======== SAMPLE 1 ========
 use the word. Do you want to be a murderer?<|endoftext|>
<|startoftext|>what happens in the universe where humans have no food?<|endoftext|>
<|startoftext|>What’s your worst, but the coolest stuff to wear?<|endoftext|>
<|startoftext|>Without naming it in the title, how is your life today different than it was in the 90’s?<|endoftext|>
<|startoftext|>People who have been to bed before 9/11. What were your very obvious signs of a real, real threat?<|endoftext|>
<|startoftext|>Whats happened to your roommate that surprised you?<|endoftext|>
<|startoftext|>Who’s the greatest teacher/student ever found o

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

## Load a Trained Model Checkpoint

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

Same as normal generate functions, except with additional parameters to handle the new tokens.

In [ ]:
gpt2.generate(sess, run_name='run1',
             length=100,
             prefix="<|startoftext|>",
             truncate="<|endoftext|>",
             include_prefix=False)

Why did you start using Reddit?


In [ ]:
gpt2.generate(sess,
              length=100,
              temperature=1.0,
              nsamples=10,
              batch_size=10,
              prefix="<|startoftext|>",
              truncate="<|endoftext|>",
              include_prefix=False
              )

What Is YOUR best advice for keeping your peace and quiet?
According to you, what is the most beautiful country in the world?
What put you in the E.R. ?
What is a question you really want answered, but not enough to waste time answering?
What was the zone of the White House where you would see the President and your pet at?
What is something you always wish someone would do for you?
What do you do to balance your busy life and save time?
As a kid, what did you want to be when you were younger?
If you could see every time the same color combination in a movie where you would be led to believe so?
What is the laziest thing you did on your cake day?


If generating in bulk, you may want to set `sample_demin=''` to remove the delimiter between each sample.

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=100,
                      temperature=1.0,
                      nsamples=100,
                      batch_size=20,
                      prefix="<|startoftext|>",
                      truncate="<|endoftext|>",
                      include_prefix=False,
                      sample_delim=''
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.